In [8]:
!lsusb

Bus 004 Device 001: ID 1d6b:0003 Linux Foundation 3.0 root hub
Bus 003 Device 002: ID 1395:0025 Sennheiser Communications Headset [PC 8]
Bus 003 Device 001: ID 1d6b:0002 Linux Foundation 2.0 root hub
Bus 002 Device 001: ID 1d6b:0003 Linux Foundation 3.0 root hub
Bus 001 Device 003: ID 04d9:1603 Holtek Semiconductor, Inc. Keyboard
Bus 001 Device 002: ID 413c:301a Dell Computer Corp. Dell MS116 USB Optical Mouse
Bus 001 Device 005: ID 067b:2303 Prolific Technology, Inc. PL2303 Serial Port
Bus 001 Device 001: ID 1d6b:0002 Linux Foundation 2.0 root hub


In [2]:
!ls -l /dev/ttyUSB*

crw-rw-rw- 1 root dialout 188, 0 Mär  2 11:56 /dev/ttyUSB0


In [24]:
!cat /etc/udev/rules.d/55-usb_serial.rules

SUBSYSTEMS=="usb", ATTRS{idVendor}=="067b", ATTRS{idProduct}=="2303", MODE:="0666"


In [3]:
import serial

## read the configuration registers

In [11]:
ser = serial.Serial('/dev/ttyUSB0',timeout=0.5)  # open serial port
print(ser.name)         # check which port was really used
print(ser.timeout)
ser.write(b'\xC1\x00\x05')     # write a string
# read the response
response = ser.read(size=10)
print(response.hex(':'))
ser.close()             # close port

/dev/ttyUSB0
0.5
c1:00:05:00:00:64:00:12


## write the configuration (persistent)

In [10]:
ser = serial.Serial('/dev/ttyUSB0',timeout=0.5)  # open serial port
print(ser.name)         # check which port was really used
print(ser.timeout)
# address = 00 00
# mode = 64 : 9600,8N1 air9.6k
# reg1 = 00
# freq = 12 : ch18=868.125 MHz
ser.write(b'\xC0\x00\x05\x00\x00\x64\x00\x12')     # command
# read the response
response = ser.read(size=10)
print(response.hex(':'))
ser.close()             # close port

/dev/ttyUSB0
0.5
c1:00:05:00:00:64:00:12


## write to one module, read from the other
(pull M0,M1 low for normal transmission mode)

In [18]:
ser = serial.Serial('/dev/ttyUSB0',timeout=0.5)  # open serial port
print(ser.name)         # check which port was really used
print(ser.timeout)
ser.write(b'hello!')     # write a string
# read the response
response = ser.read(size=10)
print(response.hex(':'))
print(response)
ser.close()   

/dev/ttyUSB0
0.5
68:65:6c:6c:6f:21
b'hello!'
